In [5]:
# This is edited from the first version to pull in a master list created by Dima and 
# use that as the ground truth for how many subjects and runs we have. Each masterlist corresponds to 
# either pilot data or analysis data. MAKE SURE TO UPDATE THIS CSV LIST WHEN DIMA IS DONE (this is not a dynamic file)

# Extra files are eliminated from the analysis in the Creating_TIER_Directory script.

# import modules
from glob import glob
import re
import os.path
from itertools import repeat
import csv
from shutil import copyfile
import shutil
import pandas as pd

pilot = 1

if pilot == 1:
    masterlist = "/om/user/rezzo/TOMLOC_INFO/tomloc_subject_info_internal.csv"
else:
    masterlist = '/om/group/saxelab/OpenAutism/data/Subject_Task_Info_Dima/subject_info_internal.csv'
    

In [6]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [11]:
root = '/om/group/saxelab/OpenAutism/data/' #+site+'/'+subject_name+'/dicom'
other_folders = ['mriqc_output', 'BIDS', '_OLD_', 'SPM_firstlevel']

all_files = []
missing_dicoms = []
missing_anat = []
missing_func_folder = []
missing_bids = []
missing_func_file = []
missing_fmriprep = []
missing_generaltsv = []
missing_mriqc_T1 = []
missing_fmriprep_smoothpreproc = []
missing_fmriprep_confounds = []
missing_mriqc_func = []
missing_firstlevel = []
missing_firstleveltime = []

with open(masterlist, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for i, line in enumerate(reader):
        if any(line) and (i != 0):
        #if (i != 0):
                subject_id = line[1] #SAX_SUB_ID
                subject_id2 = subject_id.replace("_","") #SAXSUBID
                site_id = line[3]
                task_id = line[4].replace("_","")
                run_id = line[6].split(",")

                all_files.append(subject_id+'_'+site_id+'_'+task_id)

                # check for dicom folder
                dicom_dir = os.listdir(root+site_id+'/'+subject_id+'/')
                if 'dicom' not in dicom_dir:
                    missing_dicoms.append(site_id+subject_id)
                else:
                    missing_dicoms.append("")
                        
                # check for BIDS folder
                if os.path.isdir(root+site_id+'/BIDS/sub-'+subject_id2+'/'):
                    missing_bids.append("")
                        
                    # check for BIDS tsv file
                    if not (os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/sub-'+subject_id2+"_scans.tsv")):
                        missing_generaltsv.append(sites+subjects)   
                    else:
                        missing_generaltsv.append("")  

                    # check for BIDS anat file
                    if not (os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/anat/'+'sub-'+subject_id2+'_T1w.json') and os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/anat/'+'sub-'+subject_id2+'_T1w.nii.gz')):
                        missing_anat.append(sites+subjects)
                    else:
                        missing_anat.append("")
                            
                    # check for functionals
                    missing_task = 0
                    missing_functional = 0
                    for jj in run_id:
                        jj = jj.replace(" ", "")
                        num = int(jj) -1
                        
                        # is BIDS FUNC folder present?
                        if os.path.isdir(root+site_id+'/BIDS/sub-'+subject_id2+'/func/'):
                            func_files = os.listdir(root+site_id+'/BIDS/sub-'+subject_id2+'/func/')                            
                            missing_functional = 0
                            
                            json_f = os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/func/sub-'+subject_id2+'_task-'+task_id+'_run-00'+jj+'_bold.json')
                            nifti_f = os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/func/sub-'+subject_id2+'_task-'+task_id+'_run-00'+jj+'_bold.nii.gz')
                            events_f = os.path.isfile(root+site_id+'/BIDS/sub-'+subject_id2+'/func/sub-'+subject_id2+'_task-'+task_id+'_run-00'+jj+'_events.tsv')
                            
                            if (json_f + nifti_f + events_f == 3):
                                missing_task = 0
                            else:
                                missing_task = missing_task + 1
                        else:
                            missing_task = 1
                            missing_functional = 1 
                    
                    if missing_task > 0:
                        missing_func_file.append(site_id+subject_id)
                    elif missing_task == 0:
                        missing_func_file.append("") 
                        
                    if missing_functional > 0:
                        missing_func_folder.append(site_id+subject_id)
                    elif missing_functional == 0:
                        missing_func_folder.append("") 
                                                      
                        
                else: # if BIDS folder does not exist, claim all files within missing
                    missing_bids.append(site_id+'_'+subject_id)   
                    missing_generaltsv.append(site_id+'_'+subject_id)  
                    missing_anat.append(site_id+'_'+subject_id)
                    missing_func.append(site_id+'_'+subject_id)  

                    
                # LOOK AT FMRIPREP
                missing_fmriprep_funcs = 0
                missing_fmriprep_confs = 0
                for jj in run_id:
                    jj = jj.replace(" ", "")
                    num = int(jj) -1
                
                try:  
                    listdir = os.listdir(root+site_id+'/BIDS/derivatives/fmriprep/sub-'+subject_id2+'/func/')
                    missing_fmriprep.append("")
                    preproc = "sub-"+subject_id2+"_task-"+task_id+'_run-00'+jj+"_bold_space-MNI152NLin2009cAsym_variant-smoothAROMAnonaggr_preproc.nii.gz"
                    confounds = "sub-"+subject_id2+"_task-"+task_id+'_run-00'+jj+"_bold_confounds.tsv"
                        
                    if preproc not in listdir:
                        missing_fmriprep_funcs = 1
                    if confounds not in listdir:
                        missing_fmriprep_confs = 1
                except FileNotFoundError:
                    missing_fmriprep.append(site_id+subject_id)
                    missing_fmriprep_funcs = 1
                    missing_fmriprep_confs = 1
                    
                if missing_fmriprep_funcs == 0:
                    missing_fmriprep_smoothpreproc.append("")
                else:
                    missing_fmriprep_smoothpreproc.append(site_id+subject_id)
                    
                if missing_fmriprep_confs == 0:
                    missing_fmriprep_confounds.append("")
                else:
                    missing_fmriprep_confounds.append(site_id+subject_id)
                   
                
                # LOOK AT MRIQC
                quality = 0
                
                try: 
                    tempdir = os.listdir(root+site_id+'/mriqc_output/reports/')
                    if r"sub-"+subject_id2+"_T1w.html" in tempdir:
                        missing_mriqc_T1.append("")
                    else:
                        missing_mriqc_T1.append(site_id+subject_id)
                        
                    for jj in run_id:
                        jj = jj.replace(" ", "")
                        num = int(jj) -1
                        
                        # fix below!
                        QI = "sub-"+subject_id2+"_task-"+task_id+'_run-00'+jj+"_bold.html"
                        if QI not in tempdir:
                            quality = 1
                except FileNotFoundError:
                    missing_mriqc_T1.append(site_id+subject_id)
                    quality = 1
                    
                if quality == 0:
                    missing_mriqc_func.append("")
                else:
                    missing_mriqc_func.append(site_id+subject_id) 
                    
                # look at first level anayses ######## DONE UNTIL HERE
                first_lev_error = 0
                first_level_timeseries_error = 0
                firstleveldir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard.py/'+ site_id+'/sub-'+subject_id2
                firstleveltimedir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/'+ site_id+'/sub-'+subject_id2
                sites_low = site_id.lower()

                if os.path.isdir(firstleveldir) or os.path.isdir(firstleveltimedir):
                    for jj in run_id:
                        jj = jj.replace(" ", "")
                        num = int(jj) -1        

                        firstleveldir = '/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/'+site_id+'/sub-'+subject_id2+'/'+task_id+'/model/'+'run'+jj
                        firstleveltimedir = '/om/group/saxelab/OpenAutism/Analysis/first_level_standard_timeseries.py/'+ site_id+'/sub-'+subject_id2+'/'+task_id+'/model/'+'run'+jj
                        try:
                            if len(os.listdir(firstleveldir) ) == 0:
                                first_lev_error = 1
                        except FileNotFoundError:
                            print('file/directory not found')
                            print(firstleveldir)
                            first_lev_error = 1
                        try: #added
                            if len(os.listdir(firstleveltimedir) ) == 0:
                                first_level_timeseries_error = 1
                        except FileNotFoundError:
                            print('file/directory not found')
                            print(firstleveltimedir)
                            first_level_timeseries_error = 1
                            
                else:
                    if os.path.isdir(firstleveldir) == 0:
                        #print("no FL folder")
                        first_lev_error = 1
                    if os.path.isdir(firstleveltimedir) == 0: # in addition
                        #print("no FL tmeseries folder")
                        first_level_timeseries_error = 1
                  
                if first_lev_error == 0:
                    missing_firstlevel.append("")
                else:
                    missing_firstlevel.append(site_id+subject_id) 
                    
                if first_level_timeseries_error == 0:
                    missing_firstleveltime.append("")
                else:
                    missing_firstleveltime.append(site_id+subject_id) 
            


file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti201/tomloc/model/run1
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti201/tomloc/model/run2
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti202/tomloc/model/run1
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti202/tomloc/model/run2
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti203/tomloc/model/run1
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti203/tomloc/model/run2
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti205/tomloc/model/run1
file/directory not found
/om/group/saxelab/OpenAutism/Analysis/first_level_aroma.py/TASTI2/sub-SAXtasti205/tomloc/model/run2


In [9]:


d = {'all_files': all_files, 'missing_dicoms': missing_dicoms, 'missing_bids': missing_bids,
'missing_generaltsv': missing_generaltsv, 'missing_anat':missing_anat, 'missing_func':missing_anat,
'missing_fmriprep':missing_fmriprep, 'missing_fmriprep_smoothpreproc':missing_fmriprep_smoothpreproc,
'missing_fmriprep_confounds':missing_fmriprep_confounds,'missing_mriqc_T1':missing_mriqc_T1,
'missing_mriqc_func':missing_mriqc_func, 'missing_firstlevel':missing_firstlevel, 'missing_firstleveltime': missing_firstleveltime}
    
df = pd.DataFrame(data=d, columns=['all_files', 'missing_dicoms', 'missing_bids','missing_generaltsv', 'missing_anat', 'missing_func', 
                                  'missing_fmriprep', 'missing_fmriprep_smoothpreproc', 'missing_fmriprep_confounds','missing_mriqc_T1', 'missing_mriqc_func', 'missing_firstlevel','missing_firstleveltime'])

if pilot == 1:
    df.to_csv('/om/user/rezzo/list_of_missing_tomloc_files')
else:
    df.to_csv('/om/user/rezzo/list_of_missing_files')